In [4]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
GRID_SIZE = 15
TILE_SIZE = 40
RACK_SIZE = 7
FONT = pygame.font.Font(None, 36)

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

class Tile:
    def __init__(self, letter, rect):
        self.letter = letter
        self.rect = rect
        self.dragging = False

class ScrabbleGame:
    def __init__(self):
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Scrabble Game")

        # Initialize the game board and rack
        self.board = [[' ' for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.rack = self.draw_tiles(RACK_SIZE)
        self.selected_tile = None

        # Main game loop
        self.running = True
        while self.running:
            self.handle_events()
            self.draw_board()
            self.draw_rack()
            pygame.display.flip()

        pygame.quit()

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                self.handle_mouse_down(event.pos)
            elif event.type == pygame.MOUSEMOTION:
                self.handle_mouse_motion(event.pos)
            elif event.type == pygame.MOUSEBUTTONUP:
                self.handle_mouse_up()

    def handle_mouse_down(self, mouse_pos):
        # Check if a tile on the rack is clicked
        for tile in self.rack:
            if tile.rect.collidepoint(mouse_pos):
                tile.dragging = True
                self.selected_tile = tile
                return

        # Check if a tile on the board is clicked
        for row in range(GRID_SIZE):
            for col in range(GRID_SIZE):
                tile_rect = pygame.Rect(col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE)
                if self.board[row][col] != ' ' and tile_rect.collidepoint(mouse_pos):
                    # Place the selected tile back into the rack
                    letter = self.board[row][col]
                    rect = pygame.Rect(len(self.rack) * TILE_SIZE, HEIGHT - TILE_SIZE, TILE_SIZE, TILE_SIZE)
                    self.rack.append(Tile(letter, rect))
                    self.board[row][col] = ' '  # Remove the tile from the board
                    self.selected_tile = self.rack[-1]
                    self.selected_tile.dragging = True
                    return

    def handle_mouse_motion(self, mouse_pos):
        # Move the selected tile with the mouse
        if self.selected_tile and self.selected_tile.dragging:
            self.selected_tile.rect.x, self.selected_tile.rect.y = mouse_pos

    def handle_mouse_up(self):
        # Drop the selected tile onto the board
        if self.selected_tile:
            for row in range(GRID_SIZE):
                for col in range(GRID_SIZE):
                    if self.board[row][col] == ' ' and self.selected_tile.rect.colliderect(pygame.Rect(col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE)):
                        self.board[row][col] = self.selected_tile.letter
                        self.selected_tile.dragging = False
                        self.rack.remove(self.selected_tile)  # Remove the tile from the rack
                        self.selected_tile = None
                        return
            # If not dropped on the board, return it to the rack
            self.selected_tile.dragging = False
            self.selected_tile = None


    def draw_board(self):
        self.screen.fill(WHITE)

        for row in range(GRID_SIZE):
            for col in range(GRID_SIZE):
                pygame.draw.rect(self.screen, GRAY, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE), 1)
                text = FONT.render(self.board[row][col], True, BLACK)
                self.screen.blit(text, (col * TILE_SIZE + TILE_SIZE // 3, row * TILE_SIZE + TILE_SIZE // 3))

    def draw_rack(self):
        for i, tile in enumerate(self.rack):
            pygame.draw.rect(self.screen, GRAY, tile.rect, 1)
            text = FONT.render(tile.letter, True, BLACK)
            self.screen.blit(text, (tile.rect.x + TILE_SIZE // 3, tile.rect.y + TILE_SIZE // 3))

    def draw_tiles(self, num_tiles):
        # Simulate drawing tiles from a bag (you may replace this with your own logic)
        tile_bag = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
        rack = []
        for i in range(num_tiles):
            letter = random.choice(tile_bag)
            tile_bag.remove(letter)
            rect = pygame.Rect(i * TILE_SIZE, HEIGHT - TILE_SIZE, TILE_SIZE, TILE_SIZE)
            rack.append(Tile(letter, rect))
        return rack

if __name__ == "__main__":
    ScrabbleGame()
